In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [29]:
ahu = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_ahu_automatic.csv")
chiller = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_chiller_automatic.csv")

# column_e = ['no', 'id', 'seq', 'type', 'val', 'rv', 'ri', 'sv', 'si', 'tv', 'ti', 'fire', 'temperature', 'humidity', 'cdate']
# # e = pd.read_csv('/content/drive/MyDrive/ime_project/data/porest_device_e_log.csv', names=column_e)

# column_plant = ["no", "id", "seq", "air", "monoxide", "dioxide", "pm2d5", "pm10", "temperature", "humidity", "score", "cdate"]
# plant = pd.read_csv('/content/drive/MyDrive/ime_project/data/porest_device_plant_log.csv', names=column_plant)

# # 장치 정보
# devices = pd.read_excel("/content/drive/MyDrive/ime_project/data/porest_devices.xlsx",header=1)

e & plant는 결측치가 거의 없다.
따라서 ahu와 chiller의 결측치를 추가된 데이터를 바탕으로 채우기면 하면 전처리는 끝

# ahu + chiller preprocessing

In [30]:
ahu = ahu[['ahu_no','building','power','inv','occupancy','out_temp','in_temp','zone_min_temp','cdate']]
chiller = chiller[['chiller_no','temp','power','inv','cdate']]
# e = e[['id','val','cdate']]
# plant = plant[['id','air','monoxide','dioxide','pm2d5','pm10','temperature','humidity','score','cdate']]

In [31]:
# ahu 전처리, A 구역 없앤뒤
ahu_a = ahu[ahu['building'] == 'A']
ahu_a.drop(columns=['building'], inplace=True)

# 시계열 형태로 변경
ahu_a.set_index('cdate', inplace=True)
ahu_a.index = pd.to_datetime(ahu_a.index)

ahu_group = ahu_a.groupby('ahu_no')

start_date = '2023-01-31 10:00:00'
end_date = '2023-08-01 16:00:00'

ahu_dct = {}
for name, group_df in ahu_group:
  group_df.index = group_df.index.floor('10T') # 기존의 datetime을 10분 단위로 내림
  # group_df = group_df.loc[~group_df.index.duplicated(keep='first')]
  group_df = group_df.loc[~group_df.index.duplicated(keep='first')] # 내리는 과정에서 중복 발생시 중복 제거

  date_range = pd.date_range(start=start_date, end=end_date, freq='10T')
  group_df = group_df.reindex(date_range) # 10분 단위로 row를 생성

  ahu_dct[name] = group_df

<ipython-input-31-27d029e535f5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahu_a.drop(columns=['building'], inplace=True)


In [32]:
set(ahu_a['ahu_no']) # 1~14까지의 ahu만

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}

In [33]:
ahu_dct[1].head()

,ahu_no,power,inv,occupancy,out_temp,in_temp,zone_min_temp
2023-01-31 10:00:00,1.0,1.0,30.0,8.0,24.0,\N,\N
2023-01-31 10:10:00,1.0,1.0,30.0,8.0,24.0,\N,\N
2023-01-31 10:20:00,1.0,1.0,30.0,8.0,25.0,\N,\N
2023-01-31 10:30:00,1.0,1.0,30.0,8.0,25.0,\N,\N
2023-01-31 10:40:00,1.0,1.0,30.0,8.0,25.0,\N,\N


# plus data

In [34]:
## 승호형 ##
alog = pd.read_csv("/content/drive/MyDrive/ime_project/data/porest_area_log.csv", usecols=[2, 3, 4, 6, 7],
                    names=['type', 'mode', 'text', 'cdate_0', 'cdate_1'], header=None)
alog['text'] = alog['text'].str.replace('::',',').str.replace('[','').str.replace(']',',').str.replace('\\','').str.replace(':',',')
alog = alog[(alog['type'] == 'freezer')|(alog['type'] == 'ahu')].reset_index(drop=True)
alog[['device', 'how_0', 'how_1', 'power']] = pd.DataFrame(alog.text.str.rsplit(',', 3).tolist())
alog[['name', 'day']] = pd.DataFrame(alog.device.str.rsplit(',', 1).tolist())
alog['device'] = alog['name'].str.replace(',','')
alog['cdate'] = np.where(pd.isna(alog['cdate_0']), alog['cdate_1'], alog['cdate_0'])
alog['cdate'] = pd.to_datetime(alog['cdate'])
alog['cdate'] = alog['cdate'].dt.round('10min')

alog_filtered = alog[['type', 'mode', 'device', 'how_0', 'how_1', 'power', 'day', 'cdate']]
#############

# device를 name과 number로 분리
alog_filtered['dname'] = alog_filtered['device'].str.extract('([가-힣]+)')
alog_filtered['dnumber'] = alog_filtered['device'].str.extract('(\d+)')
alog_filtered['dnumber'] = alog_filtered['dnumber'].astype('int')
alog_filtered.drop('device', axis=1, inplace=True)
# 시계열 형태로 변경
alog_filtered.set_index('cdate', inplace=True)
alog_filtered.index = pd.to_datetime(alog_filtered.index)
alog_filtered.head()

<ipython-input-34-a5f1802e7d16>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  alog['text'] = alog['text'].str.replace('::',',').str.replace('[','').str.replace(']',',').str.replace('\\','').str.replace(':',',')
<ipython-input-34-a5f1802e7d16>:6: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  alog[['device', 'how_0', 'how_1', 'power']] = pd.DataFrame(alog.text.str.rsplit(',', 3).tolist())
<ipython-input-34-a5f1802e7d16>:7: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  alog[['name', 'day']] = pd.DataFrame(alog.device.str.rsplit(',', 1).tolist())
<ipython-input-34-a5f1802e7d16>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,type,mode,how_0,how_1,power,day,dname,dnumber
cdate,,,,,,,,
2023-01-01 08:00:00,ahu,1,스케줄제어,전원,1,None,공조기,1
2023-01-01 08:00:00,ahu,1,스케줄제어,전원,1,None,공조기,3
2023-01-01 08:00:00,ahu,1,스케줄제어,전원,1,None,공조기,18
2023-01-01 08:00:00,ahu,1,스케줄제어,전원,1,None,공조기,23
2023-01-01 08:50:00,ahu,1,스케줄제어,전원,1,None,공조기,2


In [35]:
alog_filtered = alog_filtered.loc[start_date:end_date] # 시간 shorting
alog_filtered = alog_filtered[alog_filtered['how_0'] != '자동제어변경'] # 자동제어변경 제외

ahu_plus = alog_filtered[alog_filtered['type']=='ahu']
ahu_plus = ahu_plus.rename(columns={'dnumber':'ahu_no'})

chiller_plus = alog_filtered[alog_filtered['type']=='freezer']
ahu_plus.head()

,type,mode,how_0,how_1,power,day,dname,ahu_no
cdate,,,,,,,,
2023-01-31 10:20:00,ahu,0,수동제어,인버터,35Hz,None,공조기,19
2023-01-31 10:20:00,ahu,0,수동제어,인버터,35Hz,None,공조기,14
2023-01-31 10:20:00,ahu,0,수동제어,전원,1,None,공조기,14
2023-01-31 10:20:00,ahu,0,수동제어,인버터,35Hz,None,공조기,20
2023-01-31 12:20:00,ahu,0,수동제어,전원,1,None,공조기,2


In [36]:
ahu_plus = ahu_plus[ahu_plus['ahu_no']<=14] # A 빌딩안에 있는 1~14만 남김

start_date = '2023-01-31 10:00:00'
end_date = '2023-08-01 16:00:00'

In [37]:
ahu_group = ahu_plus.groupby('ahu_no')
ahu_plus_dct = {}

for name, group_df in ahu_group:
  ahu_plus_dct[name] = group_df
  ahu_plus_dct[name].loc

# ahu_no : 1

In [38]:
print(f'how_0: {set(ahu_plus_dct[1]["how_0"])} , how_1: {set(ahu_plus_dct[1]["how_1"])} , how_2: , {set(ahu_plus_dct[1]["power"])}')

how_0: {'수동제어', '스케줄제어'} , how_1: {'인버터', '전원'} , how_2: , {' 30Hz', ' 50Hz', ' 42Hz', ' 45Hz', ' 32Hz', ' 44Hz', ' 40Hz', ' 35Hz', '1', ' 48Hz', ' 46Hz', '0'}


In [39]:
ahu_man1 = ahu_plus_dct[1][ahu_plus_dct[1]['how_0']=='수동제어']
ahu_man1[ahu_man1['how_1']=='전원'].head() # power는 0,1 로 이루어져 있다.

,type,mode,how_0,how_1,power,day,dname,ahu_no
cdate,,,,,,,,
2023-01-31 13:40:00,ahu,0,수동제어,전원,0,None,공조기,1
2023-02-05 16:00:00,ahu,0,수동제어,전원,1,None,공조기,1
2023-02-05 19:50:00,ahu,0,수동제어,전원,1,None,공조기,1
2023-02-05 21:10:00,ahu,0,수동제어,전원,0,None,공조기,1
2023-02-10 11:50:00,ahu,0,수동제어,전원,1,None,공조기,1


In [40]:
ahu_man1[ahu_man1['how_1']=='인버터'].head() # power는 Hz 값으로 이루어져 있다.

,type,mode,how_0,how_1,power,day,dname,ahu_no
cdate,,,,,,,,
2023-02-01 11:40:00,ahu,0,수동제어,인버터,32Hz,None,공조기,1
2023-02-04 10:10:00,ahu,0,수동제어,인버터,35Hz,None,공조기,1
2023-02-04 15:50:00,ahu,0,수동제어,인버터,30Hz,None,공조기,1
2023-02-05 09:40:00,ahu,0,수동제어,인버터,35Hz,None,공조기,1
2023-02-05 14:10:00,ahu,0,수동제어,인버터,40Hz,None,공조기,1


In [42]:
ahu_sch1 = ahu_plus_dct[1][ahu_plus_dct[1]['how_0']=='스케줄제어']
ahu_sch1.head()

,type,mode,how_0,how_1,power,day,dname,ahu_no
cdate,,,,,,,,
2023-01-31 19:00:00,ahu,1,스케줄제어,전원,0,None,공조기,1
2023-02-01 08:50:00,ahu,1,스케줄제어,전원,1,None,공조기,1
2023-02-01 19:00:00,ahu,1,스케줄제어,전원,0,None,공조기,1
2023-02-02 08:50:00,ahu,1,스케줄제어,전원,1,None,공조기,1
2023-02-02 19:00:00,ahu,1,스케줄제어,전원,0,None,공조기,1


In [43]:
date_range = pd.date_range(start=start_date, end=end_date, freq='10T')
ahu_sch1 = ahu_sch1.reindex(date_range) # 스케줄 제어를 10분 단위로 만든다. 결측치는 NaN으로
ahu_sch1 = ahu_sch1.fillna(method='ffill') # ffill로 결측치를 채운다.
ahu_sch1.loc['2023-01-31 18:50:00':]

,type,mode,how_0,how_1,power,day,dname,ahu_no
2023-01-31 18:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-31 19:00:00,ahu,1.0,스케줄제어,전원,0,NaN,공조기,1.0
2023-01-31 19:10:00,ahu,1.0,스케줄제어,전원,0,NaN,공조기,1.0
2023-01-31 19:20:00,ahu,1.0,스케줄제어,전원,0,NaN,공조기,1.0
2023-01-31 19:30:00,ahu,1.0,스케줄제어,전원,0,NaN,공조기,1.0
...,...,...,...,...,...,...,...,...
2023-08-01 15:20:00,ahu,1.0,스케줄제어,전원,1,화,공조기,1.0
2023-08-01 15:30:00,ahu,1.0,스케줄제어,전원,1,화,공조기,1.0
2023-08-01 15:40:00,ahu,1.0,스케줄제어,전원,1,화,공조기,1.0
2023-08-01 15:50:00,ahu,1.0,스케줄제어,전원,1,화,공조기,1.0


In [44]:
ahu1 = ahu_dct[1]
ahu1.loc['2023-01-31 18:50:00':]

,ahu_no,power,inv,occupancy,out_temp,in_temp,zone_min_temp
2023-01-31 18:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-31 19:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-31 19:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-31 19:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-08-01 15:20:00,1.0,1.0,40.0,6.0,31.385,26.54,25.25
2023-08-01 15:30:00,1.0,1.0,40.0,6.0,31.405,26.63,25.25
2023-08-01 15:40:00,1.0,1.0,40.0,7.0,31.315,26.56,25.1
2023-08-01 15:50:00,1.0,1.0,40.0,7.0,31.320,26.58,25.1


In [45]:
ahu1.loc['2023-01-31 10':].head()

,ahu_no,power,inv,occupancy,out_temp,in_temp,zone_min_temp
2023-01-31 10:00:00,1.0,1.0,30.0,8.0,24.0,\N,\N
2023-01-31 10:10:00,1.0,1.0,30.0,8.0,24.0,\N,\N
2023-01-31 10:20:00,1.0,1.0,30.0,8.0,25.0,\N,\N
2023-01-31 10:30:00,1.0,1.0,30.0,8.0,25.0,\N,\N
2023-01-31 10:40:00,1.0,1.0,30.0,8.0,25.0,\N,\N


In [46]:
ahu1_com = ahu1.combine_first(ahu_sch1)
ahu1_com.loc['2023-02-01 07':].head()

,ahu_no,day,dname,how_0,how_1,in_temp,inv,mode,occupancy,out_temp,power,type,zone_min_temp
2023-02-01 07:00:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-01 07:10:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-01 07:20:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-01 07:30:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-01 07:40:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN


In [47]:
ahu1_com.loc['2023-02-01 07':].tail()

,ahu_no,day,dname,how_0,how_1,in_temp,inv,mode,occupancy,out_temp,power,type,zone_min_temp
2023-08-01 15:20:00,1.0,화,공조기,스케줄제어,전원,26.54,40.0,1.0,6.0,31.385,1.0,ahu,25.25
2023-08-01 15:30:00,1.0,화,공조기,스케줄제어,전원,26.63,40.0,1.0,6.0,31.405,1.0,ahu,25.25
2023-08-01 15:40:00,1.0,화,공조기,스케줄제어,전원,26.56,40.0,1.0,7.0,31.315,1.0,ahu,25.1
2023-08-01 15:50:00,1.0,화,공조기,스케줄제어,전원,26.58,40.0,1.0,7.0,31.320,1.0,ahu,25.1
2023-08-01 16:00:00,1.0,화,공조기,스케줄제어,전원,26.52,40.0,1.0,7.0,31.480,1.0,ahu,25.1


In [48]:
# 기존 ahu 값들은 수동제어이기 때문에 how_0를 "수동조작제어"로 바꿔준다.
ahu1_com.loc[ahu1_com['inv'].notnull(), 'how_0'] = '수동조작제어'
ahu1_com.loc['2023-02-01 07':].head()

,ahu_no,day,dname,how_0,how_1,in_temp,inv,mode,occupancy,out_temp,power,type,zone_min_temp
2023-02-01 07:00:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-01 07:10:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-01 07:20:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-01 07:30:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-01 07:40:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN


In [49]:
ahu1_com.loc['2023-02-01 07':].tail()

,ahu_no,day,dname,how_0,how_1,in_temp,inv,mode,occupancy,out_temp,power,type,zone_min_temp
2023-08-01 15:20:00,1.0,화,공조기,수동조작제어,전원,26.54,40.0,1.0,6.0,31.385,1.0,ahu,25.25
2023-08-01 15:30:00,1.0,화,공조기,수동조작제어,전원,26.63,40.0,1.0,6.0,31.405,1.0,ahu,25.25
2023-08-01 15:40:00,1.0,화,공조기,수동조작제어,전원,26.56,40.0,1.0,7.0,31.315,1.0,ahu,25.1
2023-08-01 15:50:00,1.0,화,공조기,수동조작제어,전원,26.58,40.0,1.0,7.0,31.320,1.0,ahu,25.1
2023-08-01 16:00:00,1.0,화,공조기,수동조작제어,전원,26.52,40.0,1.0,7.0,31.480,1.0,ahu,25.1


In [50]:
ahu_man1.head()

,type,mode,how_0,how_1,power,day,dname,ahu_no
cdate,,,,,,,,
2023-01-31 13:40:00,ahu,0,수동제어,전원,0,None,공조기,1
2023-02-01 11:40:00,ahu,0,수동제어,인버터,32Hz,None,공조기,1
2023-02-04 10:10:00,ahu,0,수동제어,인버터,35Hz,None,공조기,1
2023-02-04 15:50:00,ahu,0,수동제어,인버터,30Hz,None,공조기,1
2023-02-05 09:40:00,ahu,0,수동제어,인버터,35Hz,None,공조기,1


In [70]:
# how_1가 '전원'이면 power가 0,1 로 이루어져 있다. >> 이땐 그대로 power
# how_1가 '인버터'라면 power가 Hz로 이루어져 있다. >> 이떈 power를 inv값에 넣어줘야 한다.
ahu_onoff1 = ahu_man1[ahu_man1['how_1']=='전원']
ahu_inv1 = ahu_man1[ahu_man1['how_1']=='인버터']

In [77]:
ahu1_com.loc['2023-02-05 19':].head(10)

,ahu_no,day,dname,how_0,how_1,in_temp,inv,mode,occupancy,out_temp,power,type,zone_min_temp
2023-02-05 19:00:00,1.0,일,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-05 19:10:00,1.0,일,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-05 19:20:00,1.0,일,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-05 19:30:00,1.0,일,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-05 19:40:00,1.0,일,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-05 19:50:00,1.0,일,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-05 20:00:00,1.0,일,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-05 20:10:00,1.0,일,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-05 20:20:00,1.0,일,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN
2023-02-05 20:30:00,1.0,일,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,0,ahu,NaN


In [94]:
ahu_onoff1 = ahu_man1[ahu_man1['how_1']=='전원'] # 이거쓰는건 애바마인듯, 너무 불명확함
ahu_onoff1

,type,mode,how_0,how_1,power,day,dname,ahu_no
cdate,,,,,,,,
2023-01-31 13:40:00,ahu,0,수동제어,전원,0,None,공조기,1
2023-02-05 16:00:00,ahu,0,수동제어,전원,1,None,공조기,1
2023-02-05 19:50:00,ahu,0,수동제어,전원,1,None,공조기,1
2023-02-05 21:10:00,ahu,0,수동제어,전원,0,None,공조기,1
2023-02-10 11:50:00,ahu,0,수동제어,전원,1,None,공조기,1
2023-02-16 10:20:00,ahu,0,수동제어,전원,1,None,공조기,1
2023-03-13 10:30:00,ahu,0,수동제어,전원,0,None,공조기,1
2023-03-20 17:00:00,ahu,0,수동제어,전원,0,None,공조기,1
2023-03-20 17:20:00,ahu,0,수동제어,전원,1,None,공조기,1


In [91]:
ahu_onoff1 = ahu_man1[ahu_man1['how_1']=='전원']

date_range = pd.date_range(start=start_date, end=end_date, freq='10T')
ahu_onoff1 = ahu_onoff1[~ahu_onoff1.index.duplicated(keep='first')]
ahu_onoff1 = ahu_onoff1.reindex(date_range) # 스케줄 제어를 10분 단위로 만든다. 결측치는 NaN으로
ahu_onoff1 = ahu_onoff1.fillna(method='ffill') # ffill로 결측치를 채운다.
ahu_onoff1.loc['2023-02-05 19:40':].head()

,type,mode,how_0,how_1,power,day,dname,ahu_no
2023-02-05 19:40:00,ahu,0.0,수동제어,전원,1,NaN,공조기,1.0
2023-02-05 19:50:00,ahu,0.0,수동제어,전원,1,NaN,공조기,1.0
2023-02-05 20:00:00,ahu,0.0,수동제어,전원,1,NaN,공조기,1.0
2023-02-05 20:10:00,ahu,0.0,수동제어,전원,1,NaN,공조기,1.0
2023-02-05 20:20:00,ahu,0.0,수동제어,전원,1,NaN,공조기,1.0


In [98]:
ahu1_com.head()

,ahu_no,day,dname,how_0,how_1,in_temp,inv,mode,occupancy,out_temp,power,type,zone_min_temp
2023-01-31 10:00:00,1.0,NaN,NaN,수동조작제어,NaN,\N,30.0,NaN,8.0,24.0,1.0,NaN,\N
2023-01-31 10:10:00,1.0,NaN,NaN,수동조작제어,NaN,\N,30.0,NaN,8.0,24.0,1.0,NaN,\N
2023-01-31 10:20:00,1.0,NaN,NaN,수동조작제어,NaN,\N,30.0,NaN,8.0,25.0,1.0,NaN,\N
2023-01-31 10:30:00,1.0,NaN,NaN,수동조작제어,NaN,\N,30.0,NaN,8.0,25.0,1.0,NaN,\N
2023-01-31 10:40:00,1.0,NaN,NaN,수동조작제어,NaN,\N,30.0,NaN,8.0,25.0,1.0,NaN,\N


In [104]:
ahu_inv1 = ahu_man1[ahu_man1['how_1']=='인버터']
ahu_inv1.head()

,type,mode,how_0,how_1,power,day,dname,ahu_no
cdate,,,,,,,,
2023-02-01 11:40:00,ahu,0,수동제어,인버터,32Hz,None,공조기,1
2023-02-04 10:10:00,ahu,0,수동제어,인버터,35Hz,None,공조기,1
2023-02-04 15:50:00,ahu,0,수동제어,인버터,30Hz,None,공조기,1
2023-02-05 09:40:00,ahu,0,수동제어,인버터,35Hz,None,공조기,1
2023-02-05 14:10:00,ahu,0,수동제어,인버터,40Hz,None,공조기,1


In [118]:
ahu_inv1 = ahu_man1[ahu_man1['how_1']=='인버터']
ahu_inv1['method'] = ahu_inv1['how_1']

date_range = pd.date_range(start=start_date, end=end_date, freq='10T')
ahu_inv1 = ahu_inv1[~ahu_inv1.index.duplicated(keep='first')]
ahu_inv1 = ahu_inv1.reindex(date_range) # 스케줄 제어를 10분 단위로 만든다. 결측치는 NaN으로
ahu_inv1 = ahu_inv1.fillna(method='ffill') # ffill로 결측치를 채운다.
ahu_inv1['power'] = ahu_inv1['power'].str.replace('Hz', '')
ahu_inv1.loc['2023-02-01 11']

<ipython-input-118-3c09f4a425d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahu_inv1['method'] = ahu_inv1['how_1']


,type,mode,how_0,how_1,power,day,dname,ahu_no,method
2023-02-01 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 11:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 11:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 11:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 11:40:00,ahu,0.0,수동제어,인버터,32,NaN,공조기,1.0,인버터
2023-02-01 11:50:00,ahu,0.0,수동제어,인버터,32,NaN,공조기,1.0,인버터


In [119]:
ahu_inv1.rename(columns={'power':'inv'}, inplace=True)
ahu_inv1.loc['2023-02-01 11']

,type,mode,how_0,how_1,inv,day,dname,ahu_no,method
2023-02-01 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 11:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 11:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 11:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-01 11:40:00,ahu,0.0,수동제어,인버터,32,NaN,공조기,1.0,인버터
2023-02-01 11:50:00,ahu,0.0,수동제어,인버터,32,NaN,공조기,1.0,인버터


In [120]:
ahu1_com.loc['2023-02-01 11']

,ahu_no,day,dname,how_0,how_1,in_temp,inv,mode,occupancy,out_temp,power,type,zone_min_temp
2023-02-01 11:00:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,1,ahu,NaN
2023-02-01 11:10:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,1,ahu,NaN
2023-02-01 11:20:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,1,ahu,NaN
2023-02-01 11:30:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,1,ahu,NaN
2023-02-01 11:40:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,1,ahu,NaN
2023-02-01 11:50:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,1,ahu,NaN


In [127]:
ahu1_comm = ahu1_com.combine_first(ahu_inv1)
# 인버터 수동제어하는 파트(method='인버터')만 how_0, how_1 각각 바꿔준다.
ahu1_comm.loc[ahu1_comm['inv'].notnull(), 'how_0'] = '수동제어'
ahu1_comm.loc[ahu1_comm['inv'].notnull(), 'how_1'] = '인버터'
ahu1_comm.drop('method', axis=1, inplace=True)
ahu1_comm.loc['2023-02-01 11']

,ahu_no,day,dname,how_0,how_1,in_temp,inv,mode,occupancy,out_temp,power,type,zone_min_temp
2023-02-01 11:00:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,1,ahu,NaN
2023-02-01 11:10:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,1,ahu,NaN
2023-02-01 11:20:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,1,ahu,NaN
2023-02-01 11:30:00,1.0,NaN,공조기,스케줄제어,전원,NaN,NaN,1.0,NaN,NaN,1,ahu,NaN
2023-02-01 11:40:00,1.0,NaN,공조기,수동제어,인버터,NaN,32,1.0,NaN,NaN,1,ahu,NaN
2023-02-01 11:50:00,1.0,NaN,공조기,수동제어,인버터,NaN,32,1.0,NaN,NaN,1,ahu,NaN
